# Gallery Example: M[2]/M[2]/1 Multi-Class Queue

This example demonstrates a multi-class M/M/1 queue:
- **Arrivals**: Two independent Poisson processes
- **Service**: Exponential service times (same server, different rates)
- **Classes**: 2 job classes with different characteristics
- **Servers**: 1 server serving both classes
- **Scheduling**: FCFS (First-Come-First-Served)

This model is useful for analyzing systems with heterogeneous workloads.

In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_mm1_multiclass():    """Create M[2]/M[2]/1 multi-class queueing model"""    model = Network('M[2]/M[2]/1')        # Block 1: nodes    source = Source(model, 'mySource')    queue = Queue(model, 'myQueue', SchedStrategy.FCFS)    sink = Sink(model, 'mySink')        # Block 2: classes    oclass1 = OpenClass(model, 'myClass1')    source.set_arrival(oclass1, Exp(1))      # λ₁ = 1    queue.set_service(oclass1, Exp(4))       # μ₁ = 4, so service time = 0.25        oclass2 = OpenClass(model, 'myClass2')    source.set_arrival(oclass2, Exp(0.5))    # λ₂ = 0.5    queue.set_service(oclass2, Exp(4))       # μ₂ = 4, so service time = 0.25        # Block 3: topology - serial routing for both classes    P = model.init_routing_matrix()    P.add_route(oclass1, source, queue, 1.0)    P.add_route(oclass1, queue, sink, 1.0)    P.add_route(oclass2, source, queue, 1.0)    P.add_route(oclass2, queue, sink, 1.0)    model.link(P)        return model# Create the modelmodel = gallery_mm1_multiclass()

## Theoretical Analysis for Multi-Class M/M/1

For M[2]/M[2]/1 with:
- **Class 1**: λ₁ = 1, μ₁ = 4, ρ₁ = 0.25
- **Class 2**: λ₂ = 0.5, μ₂ = 4, ρ₂ = 0.125
- **Total**: λ = 1.5, ρ_total = 0.375

Under FCFS, both classes experience the same mean response time:
- **Mean Response Time**: W = 1/(μ_eff - λ_total) where μ_eff depends on class mix

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)

In [ ]:
# Analyze class-specific performance metrics
print("\n=== Class-Specific Analysis ===")

# Get detailed metrics for each class
solver = MVA(model)
avg_table = solver.avg_table()

# Extract performance metrics by class
print("Performance by class:")
for class_idx in range(model.get_number_of_classes()):
    class_name = f"Class {class_idx + 1}"
    queue_row = 1  # Queue is at index 1 (after source)
    
    # Note: The avg_table structure may vary - adjust indices as needed
    print(f"\n{class_name}:")
    print(f"  Queue metrics: {avg_table.iloc[queue_row]}")

In [ ]:
# Compare with single-class systems
print("\n=== Comparison with Single-Class Systems ===")

# Create equivalent single-class systems
def create_single_class_mm1(arrival_rate, service_rate, class_name):
    model_single = Network(f'M/M/1-{class_name}')
    source = Source(model_single, 'Source')
    queue = Queue(model_single, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_single, 'Sink')
    
    oclass = OpenClass(model_single, class_name)
    source.set_arrival(oclass, Exp(arrival_rate))
    queue.set_service(oclass, Exp(service_rate))
    
    P = model_single.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_single.link(P)
    
    return model_single

# Analyze individual class performance in isolation
model1_single = create_single_class_mm1(1.0, 4, 'Class1-Isolated')
model2_single = create_single_class_mm1(0.5, 4, 'Class2-Isolated')

solver1 = MVA(model1_single)
solver2 = MVA(model2_single)

avg_table1 = solver1.avg_table()
avg_table2 = solver2.avg_table()

print("Class 1 in isolation:")
print(avg_table1)

print("\nClass 2 in isolation:")
print(avg_table2)

print("\nNote: In multi-class FCFS, classes influence each other's performance")
print("compared to their isolated single-class equivalents.")